In [ ]:
!pip install -q langchain faiss-cpu sentence-transformers transformers


In [ ]:
from google.colab import files


In [ ]:
from google.colab import files
uploaded = files.upload()

file_name = next(iter(uploaded))
print("📂 Uploaded:", file_name)


Saving performance_reviews.json to performance_reviews.json
Saving project_assignments.csv to project_assignments.csv
Saving skill_matrix.csv to skill_matrix.csv
Saving sprint_logs.csv to sprint_logs.csv
Saving standup_notes.json to standup_notes.json
📂 Uploaded: performance_reviews.json


In [ ]:
import json
from langchain.schema import Document

documents = []

# Load standup notes
with open("standup_notes.json", "r") as f:
    standup_data = json.load(f)

for entry in standup_data:
    text = f"""
    Name: {entry.get("name")}
    Date: {entry.get("date")}
    Yesterday: {entry.get("yesterday")}
    Today: {entry.get("today")}
    Blockers: {entry.get("blockers")}
    """
    documents.append(Document(page_content=text.strip()))

# Load performance reviews
with open("performance_reviews.json", "r") as f:
    review_data = json.load(f)

for entry in review_data:
    text = f"""
    Name: {entry.get("name")}
    Review Date: {entry.get("review_date")}
    Reviewer: {entry.get("reviewer")}
    Comments: {entry.get("comments")}
    Rating: {entry.get("rating")}
    """
    documents.append(Document(page_content=text.strip()))

print(f"✅ Loaded total {len(documents)} documents")


✅ Loaded total 800 documents


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set up the chunker
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # Each chunk max ~500 characters
    chunk_overlap=50     # Overlap between chunks to preserve context
)

# Apply to all documents
chunked_docs = splitter.split_documents(documents)

print(f"✅ Split into {len(chunked_docs)} chunks")


✅ Split into 800 chunks


In [ ]:
for i, doc in enumerate(chunked_docs[:3]):
    print(f"\n--- Chunk #{i+1} ---\n{doc.page_content}")



--- Chunk #1 ---
Name: Richard Green
    Date: 2025-07-04
    Yesterday: Because important trial increase new report.
    Today: Current strategy face make.
    Blockers:

--- Chunk #2 ---
Name: Susan Marks
    Date: 2025-07-04
    Yesterday: Strong fight nice less agree.
    Today: Child feel policy.
    Blockers:

--- Chunk #3 ---
Name: Kimberly Flynn
    Date: 2025-07-04
    Yesterday: Buy program explain conference trade population agreement.
    Today: Manage wife out gun lose.
    Blockers:


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-4292522006.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunked_docs, embedding_model)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
query ="Who gave Mary Hill a review?"

In [ ]:
results = retriever.get_relevant_documents(query)

print(f"🔍 Retrieved {len(results)} relevant chunks:\n")

for i, doc in enumerate(results, 1):
    print(f"--- Chunk #{i} ---\n{doc.page_content}\n")

/tmp/ipython-input-624296230.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


🔍 Retrieved 4 relevant chunks:

--- Chunk #1 ---
Name: Mary Hill
    Review Date: 2025-07-02
    Reviewer: Daniel Burns
    Comments: Only age reveal media national feeling able security. Send suffer against and act resource. North success commercial another. Sign tough since war.
    Rating: 2.5

--- Chunk #2 ---
Name: Mary Wells
    Review Date: 2025-07-11
    Reviewer: Joyce Butler
    Comments: Community democratic no yes direction parent.
    Rating: 2.6

--- Chunk #3 ---
Name: Christine Robinson
    Review Date: 2025-06-18
    Reviewer: Riley Davenport
    Comments: Class next want instead.
    Rating: 3.6

--- Chunk #4 ---
Name: Charles Hill
    Review Date: 2025-07-21
    Reviewer: Derek Thompson
    Comments: Part read important play success.
    Rating: 4.1



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an assistant helping with performance reviews and daily standup summaries.

Use the following context to answer the user's question at the end. If the answer is not contained in the context, respond with "I don't know" — do not guess.

---------------------
{context}
---------------------

Question: {question}
Helpful Answer:"""
)


In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# Replace with better model if needed (e.g., flan-t5-small)
generator = pipeline("text-generation", model="distilgpt2", max_new_tokens=150)
llm = HuggingFacePipeline(pipeline=generator)


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-2161004273.py:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator)


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)


In [ ]:
query = "Who has less rating?"

response = qa_chain({"query": query})

print("📌 Answer:\n", response["result"])


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


📌 Answer:
 I don't know


In [ ]:
!pip install -q --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 117.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline

model_name = "google/flan-t5-large"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

# Create a generation pipeline
flan_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device=0  # Use GPU (T4)
)

# Wrap for LangChain
llm = HuggingFacePipeline(pipeline=flan_pipeline)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an assistant helping with performance reviews and daily standup summaries.

Use the following context to answer the user's question at the end. If the answer is not contained in the context, respond with "I don't know" — do not guess.

---------------------
{context}
---------------------

Question: {question}
Helpful Answer:"""
)


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)


In [ ]:
query = "Who has best rating?"

response = qa_chain({"query": query})

print("📌 Final Answer:\n", response["result"])


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


📌 Final Answer:
 I don't know


In [ ]:
for i, doc in enumerate(response["source_documents"], 1):
    print(f"\n--- Retrieved Chunk #{i} ---\n{doc.page_content}")



--- Retrieved Chunk #1 ---
Name: Kevin Hall
    Review Date: 2025-07-28
    Reviewer: Catherine Booth
    Comments: Indeed main write challenge this. Mention manager everyone dark Congress.
    Rating: 3.3

--- Retrieved Chunk #2 ---
Name: Shannon Smith
    Review Date: 2025-06-22
    Reviewer: Chad Schneider DVM
    Comments: Talk rich what truth partner measure.
    Rating: 4.4

--- Retrieved Chunk #3 ---
Name: Brandon Molina
    Review Date: 2025-06-06
    Reviewer: William Weber
    Comments: Few current like. Along public recent friend sort table. Imagine garden wall rest need major.
    Rating: 3.8

--- Retrieved Chunk #4 ---
Name: David Bruce
    Review Date: 2025-07-29
    Reviewer: Brett Keith
    Comments: So authority long kind live between.
    Rating: 3.1


In [ ]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.8 MB/s eta 0:00:00


In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr",           # Better than default similarity for mixed content
    search_kwargs={"k": 8}       # Retrieve more to improve chance of hitting rating chunks
)


In [ ]:
retriever = vectorstore.as_retriever()